In [2]:
import provdb_python.provdb_interact as pi
import provdb_python.provdb_analyze as pa
import provdb_python.provdb_counter_analyze as pca
import provdb_python.provdb_between_run_analyze as pbr
import provdb_python.provdb_viz_validate as pvv

import pymargo
from pymargo.core import Engine

from pysonata.provider import SonataProvider
from pysonata.client import SonataClient
from pysonata.admin import SonataAdmin

import sys
import os
import glob
import json
os.chdir('/home/bld/AD/develop2/benchmark_suite/func_multimodal/chimbuko/provdb')

dbs = glob.glob("provdb.*.unqlite")
nshards=len(dbs)-1 #get the number of shards in the directory (exclude the global dir)
print("Found %d shards" % nshards)
engine = Engine('na+sm', pymargo.server) 
db = pi.provDBinterface(engine, 'provdb.%d.unqlite', nshards)


Found 4 shards
Creating provider with index 1
Attaching shard 0 as provdb.0 from file provdb.0.unqlite
Attaching shard 1 as provdb.1 from file provdb.1.unqlite
Attaching shard 2 as provdb.2 from file provdb.2.unqlite
Attaching shard 3 as provdb.3 from file provdb.3.unqlite
Attaching global database as provdb.global from file provdb.global.unqlite


In [3]:
anom_coll = 'anomalies'
anom_coll_sz = db.getShard(0).size(anom_coll)
print(anom_coll_sz)

8


In [4]:
#Get a single event by index
idx=1
db.getShard(0).fetch(anom_coll,1)

{'algo_params': {'histogram': {'Bin Counts': [2,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,

In [5]:
#Get all events on the collection
ev = db.getShard(0).all(anom_coll)
print(len(ev))


8


In [21]:
#Apply a jx9 filter, returns a list
r = db.getShard(0).filter(anom_coll,'function($f){ return $f["fid"]==5;  }')  #function index == 5
print(type(r))
print(type(r[0]))
for rr in r:
    #Turn into json objects
    rrr = json.loads(rr)
    print(json.dumps(rrr,indent=2))



<class 'list'>
<class 'str'>
{
  "algo_params": {
    "histogram": {
      "Bin Counts": [
        2,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
       

In [24]:
#Rather than learning jx9, you can use genFilterFunc
                                                                                                                                                                                 
#Filters have a tuple form:  (key, comparison operator, value)                                                                                                                                                                                                              
#  comparison operator can be ==, !=, <, >                                                                                                                                                                                                                                  
#  special operator: "contains" for substring filtering                                                                                                                                                                                                                     
#Logical operations between filters are input as strings, support "(", ")", "&&", "||"                                                                                                                                                                                      
#Examples:                                                                                                                                                                                                                                                                  
#    anom = anl.filterDatabase(pdb, 'anomalies', [ ('__id','==',0) ]) #__id == 0                                                                                                                                                                                            
#                                                                                                                                                                                                                                                                           
#    anom = anl.filterDatabase(pdb, 'anomalies', [ ('exit','==',1648661837676928), "&&", ("fid","==",490) ])                                                                                                                                                                
#                                                                                                                                                                                                                                                                           
#    anom = anl.filterDatabase(pdb, 'anomalies', [ '(' , ('fid','==', 490), "||", ("fid","==",491), ')' , '&&' , ("rid","==",0) ])                                                                                                                                          
#                                                                                                                                                                                                                                                                           
#    anom = anl.filterDatabase(pdb, 'anomalies', [ ("func","contains","Kokkos") ])      

func = pa.genFilterFunc([ ('__id','==',0) ])
r = db.getShard(0).filter(anom_coll, func)
print(r)

['{"algo_params":{"histogram":{"Bin Counts":[2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2],"Bin width":996.915005,"First edge":199162.999003,"Max":398546,"Min":199163},"internal_global_threshold":2.311928},"call_stack":[{"entry":1683827518500977,"event_id":"0:1:0","exit":1683827518900084,"fid":5,"func":"the_function(int, double) [{/bld/develop2/benchmark_suite/main.C} {23,0}]","is_anomaly":true},{"entry":1683827515356351,"event_id":"0:0:2","exit":0,"fid":4,"func":"main [{/bld/develop2/benchmark_suite/main.C} {58,0}]","is_anomaly":false},{"entry":1683827515356338,"event_id":"0:0:1","exit":0,"fid":3,"func":"taupreload_main","is_anomaly":fa

In [27]:
#We also have a function based around this to filter all shards simultaneously
r = pa.filterDatabase(db,anom_coll, [ ('__id','==',0) ])
print(len(r))  #NB, if >1 shard containing data, there will be more than 1 entry with index 0! Thus only the pair
               #(shard_idx,anom_idx) is unique

1


In [27]:
#Close down cleanly
del db
engine.finalize()